In [2]:
from gensim.models import Word2Vec
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import numpy as np

/Users/kayibal/virtualenvs/ml_project/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [30]:
w2v = Word2Vec.load_word2vec_format("german.model", binary=1)

In [5]:
stop = set(stopwords.words("german"))

In [12]:
tokenizer= RegexpTokenizer(r"\w+")

In [3]:
%%bash
ls data/

category.csv
category.hdf5
question_category_train.csv
question_category_train.hdf5
question_train.csv
question_train.hdf5


In [3]:
cat = pd.read_hdf("data/category.hdf5")

In [4]:
cat.parent_id.dtype

dtype('int64')

In [35]:
main_cat = cat[cat.parent_id == 0]

In [36]:
main_cat

,category_id,parent_id,category_name,headline,title,description,meta_description,meta_tag,url,level,sort_id,use_count,active,created_at,updated_at
0,13,0,Schule,Schule | Allgemeinwissen,NaN,<p>Auch wenn man sie &bdquo;Penne&ldquo; nennt...,NaN,NaN,schule,0,56,0,1,2010-03-02 08:08:08,2010-03-31 12:19:03
1,12,0,Literatur & Sprache,Literatur & Sprache | Allgemeinwissen,NaN,"<p>Themen wie B&uuml;cher, internationale Lite...",NaN,NaN,literatur-sprache,0,41,0,1,2010-03-02 08:05:41,2010-04-01 09:28:28
2,11,0,Namensforschung,Namensforschung | Allgemeinwissen,NaN,<p>In der Kategorie &bdquo;Namensforschung&ldq...,NaN,NaN,namensforschung,0,53,0,1,2010-03-02 08:04:58,2010-03-31 13:32:21
3,7,0,Film & Musik,Film & Musik | Allgemeinwissen,NaN,<p>In der Kategorie Film &amp; Musik findest d...,NaN,NaN,film-musik,0,22,0,1,2010-03-02 07:58:55,2010-04-06 15:40:22
4,8,0,Stars & Promis,Stars & Promis | Allgemeinwissen,NaN,<p>Beim t&auml;glichen Aufschlagen der Zeitung...,NaN,NaN,stars-promis,0,64,0,1,2010-03-02 08:00:07,2010-03-31 11:21:27
5,9,0,Computer & PC,Computer & PC | Allgemeinwissen,NaN,<p>Die Kategorie &bdquo;Computer &amp; PC&ldqu...,NaN,NaN,computer-pc,0,19,0,1,2010-03-02 08:01:08,2010-04-06 15:33:48
6,10,0,Alltag,Alltag | Allgemeinwissen,NaN,<p>Der Alltag ist grob gesagt eine Aneinanderr...,NaN,NaN,alltag,0,10,0,1,2010-03-02 08:03:28,2010-04-01 17:17:42
7,14,0,Mensch & Körper,Mensch & Körper | Allgemeinwissen,NaN,<p>Der K&ouml;rper des Menschen steckt noch im...,NaN,NaN,mensch-koerper,0,46,0,1,2010-03-02 08:08:43,2010-04-01 09:16:40
8,15,0,Freizeit & Sport,Freizeit & Sport | Allgemeinwissen,NaN,<p>In der Kategorie Sport und Freizeit findest...,NaN,NaN,freizeit-sport,0,27,0,1,2010-03-02 08:09:18,2010-04-06 15:49:23
9,16,0,Wissen,Wissen | Allgemeinwissen,NaN,<p>In der folgenden Kategorie kannst du dein A...,NaN,NaN,wissen,0,69,0,1,2010-03-02 08:09:58,2010-03-31 10:38:46


In [5]:
questions = pd.read_hdf("data/question_train.hdf5")[["category_main_id", "question_id", "question", "tags"]]

In [6]:
questions = questions[questions.category_main_id != 'N']
questions["category_id"] = questions.category_main_id.astype(int)
questions.drop("category_main_id", axis=1, inplace=1)

In [7]:
questions = questions.merge(cat[["category_id", "parent_id"]], how="left").set_index("question_id")

In [8]:
questions.head()

,question,tags,category_id,parent_id
question_id,,,,
13371,Was ist THC?,THC,49,16
16360,WIEVIEL EURO UNGEFÄHR KOSTET EINE RICHTIG GROS...,"Harfe, Preis",43,15
18673,"Was bedeutet der Name ""Vassia""?","Vassia, Name, Bedeutung, Vasya, Vasiliy, Basil",26,11
4871,Hat Cristiano Ronaldo dos Santos Aveiro im Mom...,"Sport, Freundin, Fussball, Christiano",70,8
4872,Wie viel Mitarbeiter hat die otto group?,"Otto Group, Mitarbeiter, 55.000, 19 Länder",50,16


In [40]:
def clean_language(sen):
    tokens = tokenizer.tokenize(sen)
    return [w for w in tokens if w not in stop]

In [42]:
questions.iloc[:10]

,question,tags,category_id,parent_id
question_id,,,,
13371,Was ist THC?,THC,49,16
16360,WIEVIEL EURO UNGEFÄHR KOSTET EINE RICHTIG GROS...,"Harfe, Preis",43,15
18673,"Was bedeutet der Name ""Vassia""?","Vassia, Name, Bedeutung, Vasya, Vasiliy, Basil",26,11
4871,Hat Cristiano Ronaldo dos Santos Aveiro im Mom...,"Sport, Freundin, Fussball, Christiano",70,8
4872,Wie viel Mitarbeiter hat die otto group?,"Otto Group, Mitarbeiter, 55.000, 19 Länder",50,16
3288,Wie hieß die universität pgu in tiraspol?,"Universität, PGU, Tiraspol,Name",24,10
15752,Wie gross ist der sänger von billy talent Ian ...,Billy Talent,20,7
14570,Passen krebs und schütze zusammen?,"Krebs, Schütze",65,18
15256,Was bedeutet der Nane Gwendolin?,"Name, Bedeutung, Gwendolin",26,11


In [52]:
Y = questions.parent_id

In [96]:
def create_train_dataset(questions, model):
    res = []
    for idx, pid, cid, question in zip(questions.index, questions.parent_id, questions.category_id, questions.question):
        wordlist = clean_language(question)
        matrix = np.empty((len(wordlist), 304))
        matrix[:, 0] = idx
        matrix[:, 1] = pid
        matrix[:, 2] = cid
        matrix[:, 3] = np.arange(len(wordlist))
        for row, w in enumerate(wordlist):
            try:
                matrix[row, 4:] = model[w]
            except KeyError:
                pass
        train_example = pd.DataFrame(matrix)
        train_example.columns = ["question_id", "category_id", "parent_id", "wordno"] \
                                + ["dim_{}".format(i) for i in range(300)]
        res.append(train_example)

    return pd.concat(res, axis=0)

In [100]:
res = create_train_dataset(questions, w2v)

In [102]:
res

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74585 entries, 0 to 6
Columns: 304 entries, question_id to dim_299
dtypes: float64(304)
memory usage: 173.6 MB


,0
0,question_id wordno dim_0 dim_1 ...
1,question_id wordno dim_0 ...
2,question_id wordno dim_0 dim_1 ...
3,question_id wordno dim_0 dim_1 ...
4,question_id wordno dim_0 dim_1 ...
5,question_id wordno dim_0 ...
6,question_id wordno dim_0 ...
7,question_id wordno dim_0 ...
8,question_id wordno dim_0 dim_1 ...
9,question_id wordno dim_0 dim_1 ...
